In [4]:
# Núcleo de indicadores IRFCL (USD) — já validados no seu CSV
core_indicators = [
    # Estoque (ORA e composição)
    "IRFCLDT1_IRFCL65_USD_IRFCL13",   # Official reserve assets (total)
    "IRFCLDT1_IRFCL54_USD_IRFCL13",   # ORA + outros FX (liquidez ampla)
    "IRFCLDT1_IRFCL56_USD_IRFCL13",   # Ouro (incl. depósitos/swaps)
    "IRFCLDT1_IRFCL57_USD_IRFCL13",   # Posição de reservas no FMI
    "IRFCLDT1_IRFCL59_USD_IRFCL13",   # ORA: derivativos
    "IRFCLDT1_IRFCLCDCFC_USD_IRFCL13",# ORA: moeda & depósitos (FX convertível)
    "IRFCLDT1_IRFCL32_USD_IRFCL13",   # ORA: títulos
    # FX amplo adicional
    "IRFCLDT1_FR4OTH_USD_IRFCL13",    # Outros ativos FX (fora de ORA)
    "IRFCLDT1_IRFCL42_USD_IRFCL13",   # ... títulos fora de ORA
    # Ajustes de disponibilidade (memo)
    "IRFCLDT4_IRFCL150_USD_IRFCL13",  # Ativos empenhados (dentro de ORA)
    "IRFCLDT4_IRFCL69X_USD_IRFCL13",  # Títulos emprestados/repo (fora Sec. I)
    # Drenagens
    "IRFCLDT2_USD_IRFCL13",           # Predetermined drains (total)
    "IRFCLDT3_T_USD_IRFCL13",         # Contingent drains (total)
    "IRFCLDT3_SUTM_USD_IRFCL13",      # Contingent up to 1m
    "IRFCLDT3_SM1MUT3M_USD_IRFCL13",  # Contingent 1–3m
    "IRFCLDT3_SM3MUTY_USD_IRFCL13",   # Contingent 3–12m
    # Composição cambial (SDR vs não-SDR)
    "IRFCLDT4_IRFCL11_DIC_XDRB_USD_IRFCL13", # SDR basket
    "IRFCLDT4_IRFCL11_DIC_XXDR_USD_IRFCL13", # Non-SDR basket
]


In [5]:
core_sectors = [
    "S121",  # Central bank
    "S1X",   # Monetary authorities
    "S13",   # General government
    "S1311", # Central government (excl. social security)
    "S131",  # General government excluding social security
    "S12",   # Financial corporations
    "S122",  # Deposit-taking corporations (except central bank)
    "S12C",  # Deposit-taking corporations
    "S12K",  # Monetary financial institutions (MFI)
    "S123",  # Money market funds (MMFs)
    "S124",  # Non-MMF investment funds
    "S128",  # Insurance corporations
    "S129",  # Pension funds
    "S125",  # Other financial intermediaries (excl. ins./pension)
    "S126",  # Financial auxiliaries
    "S11",   # Non-financial corporations
    "S1",    # Total economy
    "S2",    # Rest of the world
    "IO",    # International organizations
    "SPE",   # Special purpose entities
]

In [6]:
important_countries = [
    # Maiores economias / benchmarks
    "USA","CHN","DEU","JPN","GBR","FRA","ITA","IND","CAN","AUS","KOR","ESP","NLD",
    # Europa ampliada
    "IRL","CHE","LUX","SWE","DNK","FIN","POL","AUT","BEL","CZE","HUN","ROU","PRT","GRC",
    # Américas (desenvolvidos + EM)
    "BRA","MEX","ARG","COL","CHL","PER","URY",
    # Ásia emergente e manufatura
    "TWN","SGP","HKG","THA","MYS","VNM","IDN","PHL",
    # Oriente Médio / petróleo
    "SAU","ARE","QAT","KWT","IRN","ISR",
    # Commodities avançados / energia
    "NOR","RUS",
    # África (representativos)
    "ZAF","EGY","NGA","GHA","KEN",
]

# Teste extra-oficial

In [7]:
import sdmx
import pandas as pd

nome_base = 'IRFCL'


# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]

params = dict(
    startPeriod="2000",
)

df_final = pd.DataFrame() 


# Abrir o arquivo para adicionar conteúdo (append)
with open("1_DICIONARIO.txt", "a", encoding="utf-8") as f:

    for country in important_countries:
        f.write("----------------------\n\n")
        for sector in core_sectors:
            for i,indicator in enumerate(core_indicators):

                # key = f'BRA+USA+CHN+IND.{indicador}.USD.A'
                key = dict(
                    COUNTRY = country,
                    INDICATOR = indicator,
                    SECTOR = sector,
                    FREQUENCY = "A",
                    # UNIT= "USDB"
                )

                # Baixar os dados
                data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

                # Converter para pandas DataFrame
                df = sdmx.to_pandas(
                    data_msg, 
                    #datetime="TIME_PERIOD"
                    )

                df = df.reset_index()

                df_final = pd.concat([df_final, df], ignore_index=True)
                if not df.empty:
                    print(f"País {country},indicador: {sector}, accounting: {indicator}")
                    f.write(f"País {country},indicador: {sector}, accounting: {indicator}\n")
                print(f"Finalizada etapa {i}") if i == 1 else print




display(df_final.head())

Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
País DEU,indicador: S1X, accounting: IRFCLDT1_IRFCL65_USD_IRFCL13
País DEU,indicador: S1X, accounting: IRFCLDT1_IRFCL54_USD_IRFCL13
Finalizada etapa 1
País DEU,indicador: S1X, accounting: IRFCLDT1_IRFCL56_USD_IRFCL13
País

KeyboardInterrupt: 

In [32]:
import sdmx
import pandas as pd

nome_base = 'IRFCL'
paises = [
    'USA',
    "BRA",
    "RUS",
    "IND",
    "CHN"
]
          
for pais in paises:
    # key = 'USA+BRA.G63G_S13_POFYGDP.A'
    key = dict(
        COUNTRY = pais,
        # BOP_ACCOUNTING_ENTRY = "NETCD_T",
        # INDICATOR = "CAB",
        # FREQUENCY = "A",
        # UNIT= "USDB"
    )

    # Cliente IMF
    client = sdmx.Client("IMF_DATA")
    base = client.dataflow(nome_base)
    nome_estrutura = list(base.structure.keys())[0]
    estrutura = base.structure[nome_estrutura]

    params = dict(
        startPeriod="2000",
    )

    # Baixar os dados
    data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

    # Converter para pandas DataFrame
    df = sdmx.to_pandas(
        data_msg, 
        #datetime="TIME_PERIOD"
        )

    df = df.reset_index()

    display(df.head())
    df.to_csv(f'../Resultados/{nome_base}_{pais}.csv', index=False)

,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value
0,USA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2019,0.0
1,USA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2020,0.0
2,USA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2021,0.0
3,USA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2022,0.0
4,USA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2023,0.0


,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value
0,BRA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2000,0.0
1,BRA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2001,0.0
2,BRA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2002,0.0
3,BRA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2003,0.0
4,BRA,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2004,0.0


,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value
0,RUS,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2004,4.242816e+10
1,RUS,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2005,6.850982e+10
2,RUS,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2006,1.092789e+11
3,RUS,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2007,1.161457e+11
4,RUS,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2008,1.609693e+10


,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value
0,IND,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2016,0.0
1,IND,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2017,0.0
2,IND,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2018,0.0
3,IND,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2019,0.0
4,IND,IRFCLDT1_IRFCL121_USD_IRFCL13,S1XS1311,A,2020,0.0


,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value
0,CHN,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2015,1.324600e+10
1,CHN,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2016,2.234600e+10
2,CHN,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2017,1.213900e+10
3,CHN,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2018,1.351300e+10
4,CHN,IRFCLDT1_IRFCL31_BOFIORC_USD_IRFCL13,S1XS1311,A,2019,6.016000e+09


In [10]:
len(df)

46997

In [24]:
indicadores = df['INDICATOR']
lista_indicadores_aceitos = set(list(indicadores))

lista = []

for item in lista_indicadores_aceitos:
    if item in core_indicators:
        lista.append(item)

print(lista)

['IRFCLDT1_IRFCL59_USD_IRFCL13', 'IRFCLDT1_IRFCL54_USD_IRFCL13', 'IRFCLDT1_IRFCL65_USD_IRFCL13', 'IRFCLDT1_IRFCL42_USD_IRFCL13', 'IRFCLDT3_SUTM_USD_IRFCL13', 'IRFCLDT4_IRFCL11_DIC_XXDR_USD_IRFCL13', 'IRFCLDT3_SM3MUTY_USD_IRFCL13', 'IRFCLDT4_IRFCL11_DIC_XDRB_USD_IRFCL13', 'IRFCLDT3_SM1MUT3M_USD_IRFCL13', 'IRFCLDT1_IRFCL56_USD_IRFCL13', 'IRFCLDT4_IRFCL69X_USD_IRFCL13', 'IRFCLDT3_T_USD_IRFCL13', 'IRFCLDT1_IRFCLCDCFC_USD_IRFCL13', 'IRFCLDT4_IRFCL150_USD_IRFCL13', 'IRFCLDT1_IRFCL57_USD_IRFCL13', 'IRFCLDT1_IRFCL32_USD_IRFCL13', 'IRFCLDT2_USD_IRFCL13']


In [28]:
print(len(lista_indicadores_aceitos))

257


In [25]:
setores = df['SECTOR']
lista_setores_aceitos = set(list(setores))

lista = []

for item in lista_setores_aceitos:
    if item in core_sectors:
        lista.append(item)

print(lista)

[]


In [27]:
print(lista_setores_aceitos)

{'S1XS1311'}


In [31]:
df.describe(exclude="number")

,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD
count,46997,46997,46997,46997,46997
unique,1,257,1,4,1613
top,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,D,2015-09-04
freq,46997,1611,46997,35538,256


In [1]:
core_indicators = [
    # 1) Nível e composição das reservas
    "IRFCLDT1_IRFCL65_USD_IRFCL13",      # Reservas oficiais (total)
    "IRFCLDT1_IRFCL54_USD_IRFCL13",      # Reservas oficiais + outros FX assets
    "IRFCLDT1_IRFCL56_USD_IRFCL13",      # Ouro
    "IRFCLDT1_IRFCL57_USD_IRFCL13",      # Posição de reservas no FMI
    "IRFCLDT1_IRFCL65_DIC_XDR_USD_IRFCL13",  # SDR (holdings) dentro das reservas
    "IRFCLDT1_IRFCL32_USD_IRFCL13",      # Títulos nas reservas
    "IRFCLDT1_IRFCLCDCFC_USD_IRFCL13",   # Moeda e depósitos nas reservas

    # 2) Drenagens de curto prazo (headline + buckets)
    "IRFCLDT2_USD_IRFCL13",                          # Drenagens líquidas de curto prazo (total)
    "IRFCLDT2_IRFCL24_SUTM_USD_IRFCL13",             # <= 1 mês (loans/securities/deposits)
    "IRFCLDT2_IRFCL24_SM1MUT3M_USD_IRFCL13",         # 1–3 meses
    "IRFCLDT2_IRFCL24_SM3MUTY_USD_IRFCL13",          # 3–12 meses

    # 2.1) Quebra por principal/juros (saídas)
    "IRFCLDT2_IRFCL26_SUTM_FO_USD_IRFCL13",          # Principal <= 1 mês
    "IRFCLDT2_IRFCL26_SM1MUT3M_FO_USD_IRFCL13",      # Principal 1–3 meses
    "IRFCLDT2_IRFCL26_SM3MUTY_FO_USD_IRFCL13",       # Principal 3–12 meses
    "IRFCLDT2_IRFCL151_SUTM_FO_USD_IRFCL13",         # Juros <= 1 mês
    "IRFCLDT2_IRFCL151_SM1MUT3M_FO_USD_IRFCL13",     # Juros 1–3 meses
    "IRFCLDT2_IRFCL151_SM3MUTY_FO_USD_IRFCL13",      # Juros 3–12 meses

    # 3) Derivativos / forwards (impacto na liquidez)
    "IRFCLDT2_IRFCL1_SUTM_IN_LP_USD_IRFCL13",        # Forwards/Futuros: entradas <= 1 mês
    "IRFCLDT2_IRFCL1_SUTM_SHP_USD_IRFCL13",          # Forwards/Futuros: saídas <= 1 mês
    "IRFCLDT4_IRFCLU97_A_USD_IRFCL13",               # Derivativos (net MTM) - memorando

    # 4) Memorando: composição por moeda e títulos em repo
    "IRFCLDT4_IRFCL11_DIC_XDRB_USD_IRFCL13",         # Cesta SDR (por moeda)
    "IRFCLDT4_IRFCL11_DIC_XXDR_USD_IRFCL13",         # Fora da cesta SDR
    "IRFCLDT4_IRFCL68_USD_IRFCL13",                  # Títulos cedidos/em repo
    "IRFCLDT4_IRFCL69X_USD_IRFCL13",                 # ...não incluídos na seção I
]

In [2]:
important_countries = [
    # Maiores economias / benchmarks
    "USA","CHN","DEU","JPN","GBR","FRA","ITA","IND","CAN","AUS","KOR","ESP","NLD",
    # Europa ampliada
    "IRL","CHE","LUX","SWE","DNK","FIN","POL","AUT","BEL","CZE","HUN","ROU","PRT","GRC",
    # Américas (desenvolvidos + EM)
    "BRA","MEX","ARG","COL","CHL","PER","URY",
    # Ásia emergente e manufatura
    "TWN","SGP","HKG","THA","MYS","VNM","IDN","PHL",
    # Oriente Médio / petróleo
    "SAU","ARE","QAT","KWT","IRN","ISR",
    # Commodities avançados / energia
    "NOR","RUS",
    # África (representativos)
    "ZAF","EGY","NGA","GHA","KEN",
]

In [3]:
import sdmx
import pandas as pd

nome_base = 'IRFCL'


# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]

params = dict(
    startPeriod="2000",
)

df_final = pd.DataFrame() 


# Abrir o arquivo para adicionar conteúdo (append)
with open("1_DICIONARIO.txt", "a", encoding="utf-8") as f:

    for country in important_countries:
        f.write("----------------------\n\n")
        for i,core_indicator in enumerate(core_indicators):
            # key = f'BRA+USA+CHN+IND.{indicador}.USD.A'
            key = dict(
                COUNTRY = country,
                # BOP_ACCOUNTING_ENTRY = ac,
                INDICATOR = core_indicator,
                FREQUENCY = "Q",
                # UNIT= "USDB"
            )

            # Baixar os dados
            data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

            # Converter para pandas DataFrame
            df = sdmx.to_pandas(
                data_msg, 
                #datetime="TIME_PERIOD"
                )

            df = df.reset_index()

            df_final = pd.concat([df_final, df], ignore_index=True)
            if not df.empty:
                print(f"País {country},indicador: {core_indicator}")
                f.write(f"\nPaís {country},indicador: {core_indicator}")
            print(f"Finalizada etapa {i}") if i == 1 else print




display(df_final.head())
df_final.to_csv(f"../Resultados/{nome_base}.csv", index=False)

País USA,indicador: IRFCLDT1_IRFCL65_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL54_USD_IRFCL13
Finalizada etapa 1
País USA,indicador: IRFCLDT1_IRFCL56_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL57_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL65_DIC_XDR_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCL32_USD_IRFCL13
País USA,indicador: IRFCLDT1_IRFCLCDCFC_USD_IRFCL13
País USA,indicador: IRFCLDT2_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL24_SUTM_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL24_SM1MUT3M_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL24_SM3MUTY_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL26_SUTM_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL26_SM1MUT3M_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL26_SM3MUTY_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL151_SUTM_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL151_SM1MUT3M_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL151_SM3MUTY_FO_USD_IRFCL13
País USA,indicador: IRFCLDT2_IRFCL1_SUTM_IN_LP_USD_IRFCL13

,COUNTRY,INDICATOR,SECTOR,FREQUENCY,TIME_PERIOD,value,index
0,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2000-Q4,6.693000e+10,NaN
1,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q1,6.371900e+10,NaN
2,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q2,6.435500e+10,NaN
3,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q3,7.038200e+10,NaN
4,USA,IRFCLDT1_IRFCL65_USD_IRFCL13,S1XS1311,Q,2001-Q4,6.862100e+10,NaN
